In [4]:
from bs4 import BeautifulSoup
import lxml
import requests
from PIL import Image
import io
import re
import os
from urllib.request import Request, urlopen
import pandas as pd

In [6]:
all_mss_df = pd.read_csv("all_syriac_mss data.csv", index_col = 'ms name')
all_mss_df.drop("Unnamed: 0", axis=1, inplace=True)
all_mss_df.head()

,language,date,contents
ms name,,,
ACA 00001,Syriac,"1,768 CE",['Book of Directions and admonitions\nܟܬܒܐ ܕܗܘ...
ACA 00002,"Neo-Aramaic, Syriac","1,766 CE","['Evangelion (Syriac Rite, Eastern)\nRubrics a..."
ACA 00003,"Garshuni, Syriac","1,824 CE","['Book of Ordination\nܟܬܒܐ ܕܣܝܡܝܕܐ', 'ܟܬܒܐ ܕܣܝ..."
ACE 00001,Syriac,"1,700 CE","['Pentateuch (Genesis, Exodus, Leviticus, Numb..."
ACE 00002,Syriac,"1,882 CE",['Selections of the Old Testament\nܟܬܒܐ ܕܡܬܩܪܐ...


In [5]:
link = "http://syri.ac/digimss/faceted?items_per_page=100"
# field = "field_collection"
# tag = "beinecke-3"
# link = f"http://syri.ac/digimss/faceted/{field}/{tag}?items_per_page=100"

In [6]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
#soup.findAll("li", {"leaf"}) 
#a class="facetapi-inactive" href="/digimss/faceted/field_collection/beinecke-3?items_per_page=100
all_a = soup.findAll("a", {"facetapi-inactive"})
hrefs = []
for a in all_a:
    href = a.get('href')
    hrefs.append(href)
hrefs

['/digimss/faceted/field_language_tag/arabic-4?items_per_page=100',
 '/digimss/faceted/field_language_tag/armenian-7?items_per_page=100',
 '/digimss/faceted/field_language_tag/armenian-syriac-script-8?items_per_page=100',
 '/digimss/faceted/field_language_tag/chaldean-45?items_per_page=100',
 '/digimss/faceted/field_language_tag/chinese-770?items_per_page=100',
 '/digimss/faceted/field_language_tag/christian-palestinian-aramaic-44?items_per_page=100',
 '/digimss/faceted/field_language_tag/coptic-1044?items_per_page=100',
 '/digimss/faceted/field_language_tag/english-723?items_per_page=100',
 '/digimss/faceted/field_language_tag/french-1026?items_per_page=100',
 '/digimss/faceted/field_language_tag/garshuni-6?items_per_page=100',
 '/digimss/faceted/field_language_tag/geez-1070?items_per_page=100',
 '/digimss/faceted/field_language_tag/georgian-9?items_per_page=100',
 '/digimss/faceted/field_language_tag/greek-43?items_per_page=100',
 '/digimss/faceted/field_language_tag/hebrew-716?items

In [14]:
href = "/digimss/faceted/field_language_tag/latin-39?items_per_page=100"
link = f"http://syri.ac{href}"
page = 0
full_link = f"{link}&page={page}"
full_link

'http://syri.ac/digimss/faceted/field_language_tag/latin-39?items_per_page=100&page=0'

In [28]:
link_ptrn =  re.compile(r'(http://syri.ac/digimss/faceted/field_)(?P<tag>.+?)(/)(?P<content>.+?)(\?items_per_page=100&page=)(?P<pages>\d+)', re.DOTALL | re.MULTILINE | re.UNICODE)
tr = re.search(link_ptrn, full_link)
print(tr.group("tag"))
print(tr.group("content"))
print(tr.group("pages"))

language_tag
latin-39
0


In [ ]:
all_dfs = []
for href in hrefs:
    df = collect_href(href)
    all_dfs.append(df)     

finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection
finished page collection


In [63]:
def collect_href(href):
    link_ptrn =  re.compile(r'(?P<start>http://syri.ac/digimss/faceted/field_)(?P<tag>.+?)(/)(?P<content>.+?)(\?items_per_page=100&page=)(?P<pages>\d+)', re.DOTALL | re.MULTILINE | re.UNICODE)
    all_mss_data = []
    page = 0
    link = f"http://syri.ac{href}"
    full_link = f"{link}&page={page}"
    all_mss_data.append(get_mss(link))
    next_try = next_link(full_link, page)
    while next_try is not None:
        temp_next_link = next_try
        all_mss_data.append(get_mss(temp_next_link))
        page = page +1
        next_try = next_link(temp_next_link, page)
    df = pd.DataFrame(all_mss_data)
    tr = re.search(link_ptrn, link)
    df[tr.group("tag")] = tr.group("content")
    print("finished href collection")
    return df
        
        
def next_link(link, page):
    tr = re.search(link_ptrn, link)
    current = tr.group("pages")
    try:
        link = f"{link}&page={current+1}"
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        print(link)
        return link
    except:
        return None
        
def get_mss(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    mss_data = []
    page_rows = soup.findAll("tr")
    for row in page_rows[1:]:
        ms_name = row.find("td").find('a').contents[0]
        mss_data.append({'ms name':ms_name})
    print("finished page collection")
    return mss_data

In [49]:
tr = re.search(link_ptrn, link)
tr.group("tag")

'content_key_words_other_ge'

In [ ]:
page_rows = soup.findAll("tr")
for row in page_rows[1:]:
    mss = {}
    ms_name = row.find("td").find('a').contents[0]
    
    if more pages... #t[0].find('li') is not None:
        repeat for every_page
    else:
        mss.update({'ms name':ms_name,
       'contents':ms_contents
               })
    mss_data.append(mss)

In [37]:
link_pages = [i for i in range(26)]

In [38]:
#mss_data = []
for page in link_pages:    
    link = f'http://syri.ac/digimss/faceted/field_collection/field_content_key_words_other_ge/ascetion-558?items_per_page=100&page={page}'
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_rows = soup.findAll("tr")
    for row in page_rows[1:]:
        mss = {}
        ms_name = row.find("td").find('a').contents[0]
        ms_lang = row.find('td',{'class': 'views-field-field-language'}).contents[0].strip()
        ms_date = row.find('td',{'class': 'views-field-field-ms-approximate-date'}).contents[0].strip()
        ms_contents = []
        t = row.find_all('td',{'class': 'views-field-body'})
        if t[0].find('li') is not None:
            for li in t[0].find_all('li'):
                ms_contents.append(li.text.strip())
        else:
            ms_contents.append(t[0].find('p').contents[0].strip())

        #ms_link = row.findAll(".views-field-field-link-to-digitized-manuscri a").xpath("@href").extract()
        mss.update({'ms name':ms_name,
           'language':ms_lang,
           'date':ms_date,
           'contents':ms_contents,
           #'link':ms_link
                   })
        mss_data.append(mss)

In [39]:
len(mss_data)

217

In [40]:
df = pd.DataFrame(mss_data)
df.head(10)

,ms name,language,date,contents
0,Borg. sir. 128,Syriac,"1,720 CE","[Binding, f. 1r-67v History of Mary, f. 67v-71..."
1,Vat. sir. 37,"Garshuni, Syriac","1,627 CE","[Binding, f. 1v-57r Burial service, Garshuni a..."
2,Vat. sir. 117,Syriac,700 CE,"[Binding, f. 1r Contents, f. 1v-9v Jacob of Sa..."
3,Vat. sir. 160,Syriac,900 CE,"[Binding, f. 1v-79v Life and Letters of Simeon..."
4,Vat. sir. 161,Syriac,800 CE,"[Binding, f. 1r-20r Two Sermons on the Persian..."
5,Mingana Syriac 112,Syriac,"1,905 CE",[Moshe bar Kepha\nf. 1v Annunciation of Zechar...
6,Mingana Syriac 148,Syriac,"1,613 CE","[f. 1v-2v, 5, 7r-13v Moses Karkhāya (from Kark..."
7,Mingana Syriac 343,Syriac,"1,350 CE","[f. 1-93r, 103v-242v The Commentary of Dionysi..."
8,Vat. sir. 467,Syriac,"1,186 CE","[Binding, f. 1v-222r Life of Yusuf Busnaya, f...."
9,Vat. sir. 584,Syriac,"1,899 CE","[Binding, f. 1v-56v History of Rabban bar 'Idt..."


In [41]:
df.to_excel('syriac_mss_theology (martyrologies, discourses, ascetions) data.xlsx', encoding='utf-8')

In [8]:
mss_data[290]

{'ms name': 'ASOM 00022 007',
 'language': 'Syriac',
 'date': 'Unknown',
 'contents': ['Anaphora of Mar James, the brother of our Lord (Syriac Rite, Western)\nTitle NS: ܐܢܐܦܘܪܐ ܕܡܪܝ ܝܥܩܘܒ ܐܚܘܗܝ ܕܡܪܢ',
  'Title NS: ܐܢܐܦܘܪܐ ܕܡܪܝ ܝܥܩܘܒ ܐܚܘܗܝ ܕܡܪܢ']}

In [17]:
#link_pages = [i for i in range(2)]
link = f'http://syri.ac/digimss/faceted/&items_per_page=100&page=0'
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')
page_rows = soup.findAll("tr")

In [ ]:
page_rows(1)

In [ ]:
for row in page_rows[1:]:
    mss = {}
    ms_name = row.find("td").find('a').contents[0]
    ms_lang = row.find('td',{'class': 'views-field-field-language'}).contents[0].strip()
    ms_date = row.find('td',{'class': 'views-field-field-ms-approximate-date'}).contents[0].strip()
    ms_contents = []
    t = row.find_all('td',{'class': 'views-field-body'})
    if t[0].find('li') is not None:
        for li in t[0].find_all('li'):
            ms_contents.append(li.text.strip())
    else:
        ms_contents.append(t[0].find('p').contents[0].strip())

    #ms_link = row.findAll(".views-field-field-link-to-digitized-manuscri a").xpath("@href").extract()
    mss.update({'ms name':ms_name,
       'language':ms_lang,
       'date':ms_date,
       'contents':ms_contents,
       #'link':ms_link
               })
    mss_data.append(mss)